# Plan Recognition Demo
## By Loren Champlin

This is a demo of my plan recognition prototype algorithm. 

First, we assume that there is an agent who is trying to complete some objective (e.g, Search and Rescue) and that they are following a plan (i.e list of grounded actions) that accomplishes this objective. This plan is generated by some type of planner (e.g, Hierarchical Task Network), one that we engineered. We also assume that there are a number of different strategies the agent can adopt to complete their objective. These different strategies are represented as top-level tasks in the planner. Given that we observe the sequence of actions an agent is taking as well as the sequence of states that procedes each action (i.e, state, action pairs), the goal of our algorithm is to infer what top-level task was used to generate the agent's plan and thus infer what strategy they are using to complete their objective. 

The algorithm is implemented using a bayesian model. Let $S$ be a categorical random variable representing the possible strategies/top-level tasks and let $O_t = (s_0,a_0), (s_1,a_1), ... , (s_t, a_t)$ where $s_i$ and $a_i$ is the state and action at time $t$. Note that $s_i$ actually happens before $a_i$ (i.e, it is the state that precedes the action), but for simplicity we will assume we observe them at the same time. With these variables defined we can use Baye's Rule to compute,

$P(S | O_t) = \frac{P(O_t | S) P(S)}{P(O_t)}$

Notice that $P(O_t)$ is just $\sum{S}P(O_t | S) P(S)$, in other words we can compute $P(O_t | S) P(S)$ for each value of S and then just normalize the resulting probabilities to add up to 1 by dividing each one by $\sum{S}P(O_t | S) P(S)$. 

The only challenge in this equation is computing the likelihood $P(O_t | S)$. However we can estimate the likelihood by assuming that our planner is complete (i.e, it can simulate the distribution of all possible plans the agent can execute). Therefore we can generate plans for each value of $S$ and see if $O_t$ matches any of the plan traces (i.e, plans with both states and actions, just like our observations) up to time $t$. This matching process allows us to compute, 

$P(O_t | S) \approx \frac{\text{number of plans from $S$ that match $O_t$}}{\text{total number of plans from $S$}}$ 

In [1]:
from plan_recognition import upload_plans, posterior_dist

This next line uploads all possible plan traces from the Simple Schedule Domain. The domain has the agent executing a series of actions such as going to school, going to work, doing chores, etc. This schedule of tasks is determined by what day (Monday, Tuesday, Wednesday, Thursday, or Friday) it is. The day serves as the strategy or top-level task in this case. 

In [2]:
ptts = upload_plans("simple-schedule-plan-traces.json")

For demonstration purposes, I grab one of the generated plans as the sequence of observed state, action pairs of the agent. 

In [3]:
p = 0
obs = []
for i in ptts[p].getleafNodes():
    obs.append((i.state,i.task))

At this point, we will assume that there is just one observation as seen below. 

In [4]:
print("Observations \n")
print("True Top-level task: ",ptts[p].task,"\n")
for i,j in enumerate(obs[:1]):
    print("time: ",i,", state: ",j[0], ", task: ",j[1], "\n")

Observations 

True Top-level task:  MONDAY ME 

time:  0 , state:  ((FOUND-MOVIE) (HAVE-HOMEWORK) (HUMAN ME) (NEED-GROCERIES) (RAINING) (WORK-TODAY)) , task:  !GO-TO-SCHOOL ME 



Here the prior distribution of days is uniform (i.e, each day is equally as likely). 

In [5]:
cats = [{"task": "MONDAY ME", "prior": 1/5},{"task": "TUESDAY ME", "prior": 1/5},
       {"task": "WEDNESDAY ME", "prior": 1/5},{"task": "THURSDAY ME", "prior": 1/5},
       {"task": "FRIDAY ME", "prior": 1/5}]
post = posterior_dist(obs[:1],cats,ptts)

With just one observation, we can see that all other days aside from Monday and Wednesday have been eliminated as possible top-level tasks. 

In [6]:
for i in post:
    print("Top-level task: ",i["task"], ", Posterior Belief: ",i["posterior"],"\n")

Top-level task:  MONDAY ME , Posterior Belief:  0.5 

Top-level task:  TUESDAY ME , Posterior Belief:  0.0 

Top-level task:  WEDNESDAY ME , Posterior Belief:  0.5 

Top-level task:  THURSDAY ME , Posterior Belief:  0.0 

Top-level task:  FRIDAY ME , Posterior Belief:  0.0 



Here is the same observation sequence with now two state, action pairs. 

In [7]:
print("Observations \n")
print("True Top-level task: ",ptts[p].task,"\n")
for i,j in enumerate(obs[:2]):
    print("time: ",i,", state: ",j[0], ", task: ",j[1], "\n")

Observations 

True Top-level task:  MONDAY ME 

time:  0 , state:  ((FOUND-MOVIE) (HAVE-HOMEWORK) (HUMAN ME) (NEED-GROCERIES) (RAINING) (WORK-TODAY)) , task:  !GO-TO-SCHOOL ME 

time:  1 , state:  ((FOUND-MOVIE) (HAVE-HOMEWORK) (HUMAN ME) (NEED-GROCERIES) (RAINING) (WENT-TO-SCHOOL) (WORK-TODAY)) , task:  !GO-TO-WORK ME 



The same prior was kept here.

In [8]:
cats = [{"task": "MONDAY ME", "prior": 1/5},{"task": "TUESDAY ME", "prior": 1/5},
       {"task": "WEDNESDAY ME", "prior": 1/5},{"task": "THURSDAY ME", "prior": 1/5},
       {"task": "FRIDAY ME", "prior": 1/5}]
post = posterior_dist(obs[:2],cats,ptts)

With just two observations, we can confirm that the top-level task is Monday. This of course might not be the case with a more complicated domain, which may take several observations to reach a posterior belief close to or at 1 for any specific top-level task. 

In [9]:
for i in post:
    print("Top-level task: ",i["task"], ", Posterior Belief: ",i["posterior"], "\n")

Top-level task:  MONDAY ME , Posterior Belief:  1.0 

Top-level task:  TUESDAY ME , Posterior Belief:  0.0 

Top-level task:  WEDNESDAY ME , Posterior Belief:  0.0 

Top-level task:  THURSDAY ME , Posterior Belief:  0.0 

Top-level task:  FRIDAY ME , Posterior Belief:  0.0 



Here the observation sequence is changed and we give the bayesian model the full sequence. 

In [10]:
p = 103
obs = []
for i in ptts[p].getleafNodes():
    obs.append((i.state,i.task))

In [11]:
print("Observations \n")
print("True Top-level task: ",ptts[p].task, "\n")
for i,j in enumerate(obs[:4]):
    print("time: ",i,", state: ",j[0], ", task: ",j[1],"\n")

Observations 

True Top-level task:  THURSDAY ME 

time:  0 , state:  ((HAVE-HOMEWORK) (HUMAN ME) (RAINING)) , task:  !GO-TO-SCHOOL ME 

time:  1 , state:  ((HAVE-HOMEWORK) (HUMAN ME) (RAINING) (WENT-TO-SCHOOL)) , task:  !DO-HOMEWORK ME 

time:  2 , state:  ((DID-HOMEWORK) (HUMAN ME) (RAINING) (WENT-TO-SCHOOL)) , task:  !DO-CHORES ME 

time:  3 , state:  ((DID-CHORES) (DID-HOMEWORK) (HUMAN ME) (RAINING) (WENT-TO-SCHOOL)) , task:  !PLAY-VIDEOGAMES ME 



In [12]:
cats = [{"task": "MONDAY ME", "prior": 1/5},{"task": "TUESDAY ME", "prior": 1/5},
       {"task": "WEDNESDAY ME", "prior": 1/5},{"task": "THURSDAY ME", "prior": 1/5},
       {"task": "FRIDAY ME", "prior": 1/5}]
post = posterior_dist(obs[:4],cats,ptts)

Notice under the uniform prior, despite having the full observation sequence we don't overwhelming belief that the top-level task was Thursday (the true task) rather than Wednesday. It's obvious that both Thursday and Wednesday under this initial state produce the same plan and with any of the days being equally as likely in this case, we cannot rule out one over the other. 

In [13]:
for i in post:
    print("Top-level task: ",i["task"], ", Posterior Belief: ",i["posterior"],"\n")

Top-level task:  MONDAY ME , Posterior Belief:  0.0 

Top-level task:  TUESDAY ME , Posterior Belief:  0.0 

Top-level task:  WEDNESDAY ME , Posterior Belief:  0.5 

Top-level task:  THURSDAY ME , Posterior Belief:  0.5 

Top-level task:  FRIDAY ME , Posterior Belief:  0.0 



However it is possible that our prior belief could give us evidence to shift our posterior belief in favor of Thursday rather than Wednesday. This is shown below. 

In [14]:
cats = [{"task": "MONDAY ME", "prior": 3/20},{"task": "TUESDAY ME", "prior": 3/20},
       {"task": "WEDNESDAY ME", "prior": 3/20},{"task": "THURSDAY ME", "prior": 2/5},
       {"task": "FRIDAY ME", "prior": 3/20}]
post = posterior_dist(obs[:4],cats,ptts)

While we are not getting a posterior belief of 1 like in other cases, we still achieved a majority posterior belief in the day being Thursday. Of ocourse a different prior could yield different results with Wednesday having the larger posterior belief instead. 

In [15]:
for i in post:
    print("Top-level task: ",i["task"], ", Posterior Belief: ",i["posterior"],"\n")

Top-level task:  MONDAY ME , Posterior Belief:  0.0 

Top-level task:  TUESDAY ME , Posterior Belief:  0.0 

Top-level task:  WEDNESDAY ME , Posterior Belief:  0.2727272727272727 

Top-level task:  THURSDAY ME , Posterior Belief:  0.7272727272727273 

Top-level task:  FRIDAY ME , Posterior Belief:  0.0 



# Issues to address

There are two main issues to address with this prototype algorithm. 

The first issue is that the algorithm assume that the state of the domain is fully observable. For example, in the SAR missions we don't know where the victims are before hand. Therefore under a deterministic planner, we would not be able to generate actual plans. 

One fix is to have uncertainty in the planner (e.g, probablistic effects). In the case where there is uncertainty in the planner (i.e, the same initial state and top-level task can produce different plans), then we merely need to generate plans for each initial state and top-level task combo multiple times until we get a large sampling of the most likely plans. I suspect that this would cause only minor errors when estimating the likelihood of the bayesian model. 

Alternatively the planner could be deterministic and we could instead sample initial states (and assume those are fully observable states). In other words for a domain like the SAR mission, we would sample where the victims might be. To get a decent likelihood estimate in either case, we would need to somewhat accurately estimate certain distributions such as the initial state in the alternative case. 

Another fix for this issue is to generate a plan based off of a reduce but fully observable state reflecting the current state of for example the SAR mission. 

The second issue is that algorithm assumes an infallible execution of a plan generated by the corresponding planner. What this means in terms of the SAR mission is that the agent will never diverge from the strategy they using and will use the strategy perfectly. So if for example the player in the SAR mission has adopted the yellow-first strategy and for some reason they make just one action not charactistic of that strategy (e.g, triage a green victim in the dark bathroom before 5 minutes is up), then the plan recognition algorithm will rule out that strategy even if the player continues to follow that strategy for the rest of the game. 

One fix to this strategy is to have a moving window when doing the observation/plan matching. So instead of always matching the observations to possible plans from time 0, match them starting at some time i that increase with each new observation. This way a slight divergence from a strategy will only confuse our algorithm for a short time. 

Another fix is to assign a similarity measure between state, action pair sequences where a similarity of 1 means that the sequences are identical and a similarity of 0 means that they have nothing in common. Then we could estimate the bayesian model's likelihood by summing the similarity measures of each plan to the observed sequence. In other words,

$P(O_t | S) \approx \frac{\text{sum of similarity measures between plans from $S$ and $O_t$}}{\text{total number of plans from $S$}}$

I suspect that this would make it so that each value of S has at least some probability greater than 0 of generating any observation sequence. 